<a href="https://colab.research.google.com/github/JALVARADORUIZ/TFM_UB/blob/main/TFM_modelo_svm_variety.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **PREPARACIÓN DEL ENTORNO**


In [8]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle
from tqdm import tqdm
tqdm.pandas()

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle
from tqdm import tqdm

In [6]:
tqdm.pandas()

#### **PREPROCESADO DE IMÁGENES**


In [74]:
data = '/content/drive/MyDrive/DATAWINE/df_img_processed2.csv'

df = pd.read_csv(data)

Analisis de las etiquetas asociadas a cada una de las imagenes:

In [10]:
df.head(5)

,winery,product,region,country_code,rating,variety,price_usd,image,continente,country_code_clean,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,cathedral ridge,cathedral ridge bangsund vineyard pinot noir 2006,columbia valley oregon,us,3.5,pinot noir,163.000000,vintage-images/MBo5T9ZQQQGbzsRmVORp-w_pl_480x6...,north america,united states,...,0.000000,2.723852,0.112117,1.256278,0.0,0.0,0.688721,0.0,0.0,0.641749
1,penfolds,penfolds rwt shiraz 2005,barossa valley,au,4.5,shiraz syrah,169.950000,vintage-images/dDyTwzIfQC68MCUgjmYWpw_pl_480x6...,oceania,australia,...,1.020779,0.000000,0.000000,2.622155,0.0,0.0,0.290430,0.0,0.0,0.000000
2,marques de riscal,marques de riscal frank gehry selection 2001,rioja,es,5.0,tempranillo,357.196682,vintage-images/t44NPpbUT6uy0vRt4VD2RQ_pl_480x6...,europe,spain,...,0.000000,0.555558,0.000000,0.656157,0.0,0.0,0.000000,0.0,0.0,1.079562
3,wyndham,wyndham pinot noir bin 333 2007,south eastern australia,au,4.0,pinot noir,10.990000,vintage-images/v1RahJe2QQyp4oAFOv90bA_pl_480x6...,oceania,australia,...,0.000000,1.728447,0.707242,0.000000,0.0,0.0,0.000000,0.0,0.0,1.881952
4,louis jadot,louis jadot chevalier montrachet grand cru 2006,chevalier montrachet grand cru,fr,5.0,chardonnay,309.990000,vintage-images/VlMihlZUQE6US7fr09w5ZA_pl_480x6...,europe,france,...,0.000000,2.167833,0.000000,0.000000,0.0,0.0,0.400719,0.0,0.0,2.144384


In [75]:
df_variety = df['variety'].value_counts().rename_axis('variety').to_frame('counts')
df_variety

,counts
variety,
cabernet sauvignon,2215
chardonnay,1030
shiraz syrah,802
pinot noir,714
merlot,519
...,...
symphony,1
moscatel,1
cencibel,1


In [76]:
df_variety_selected = df_variety[:15]
df_variety_selected

,counts
variety,
cabernet sauvignon,2215
chardonnay,1030
shiraz syrah,802
pinot noir,714
merlot,519
sangiovese,431
nebbiolo,339
sauvignon blanc,300
tempranillo,281


In [77]:
list_variety = list(df_variety_selected.index)

def transform_variety(variety, list_variety):
  if variety in list_variety:
    return variety
  else:
    return "other"

df['variety'] = df.progress_apply(lambda x:transform_variety(x.variety,list_variety), axis=1)

100%|██████████| 9878/9878 [00:02<00:00, 3392.97it/s]


In [78]:
le = LabelEncoder()
variedad = le.fit_transform(df['variety'])

In [79]:
X = df.iloc[:,15:].values
y = variedad

**1. SVM con datos originales (4096 features)**

In [17]:
from sklearn.svm import SVC

In [20]:
X.shape, y.shape

((9878, 4096), (9878,))

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [22]:
# Con parametros por defecto
classifier_scv_original = SVC(kernel='rbf', random_state=32)

In [23]:
classifier_scv_original.fit(X_train, y_train)


SVC(random_state=32)

In [24]:
y_pred_original = classifier_scv_original.predict(X_test)

In [25]:
result_original = confusion_matrix(y_test, y_pred_original)

print('Confusion Matrix Clasificador SVC original')

print(result_original)

print('Accuracy:', accuracy_score(y_test, y_pred_original))

Confusion Matrix Clasificador SVC original
[[ 13  36   4   0   0   0   0   7   9   0   0   0   0   0   0   0]
 [  0 629   3   0   0   1   0  80   4   0   1   0   2   0   0   0]
 [  0   9 173   0   0   0   0 104   9   0   0   0   3   0   0   0]
 [  0   9   1   6   0   0   0  14  10   0   0   0  10   0   0   0]
 [  0  47   0   0   4   0   0   8   0   0   0   0   0   0   0   0]
 [  0 120   0   0   0  27   0  25   1   0   1   0   0   0   0   0]
 [  0  44   1   0   0   0  16  31   7   0   0   0   2   0   0   0]
 [  0 127  65   0   0   0   0 328  33   0   0   0   7   0   2   0]
 [  0   9  18   0   0   0   0  68 118   0   0   0  11   0   0   0]
 [  0   1  11   0   0   0   0  30   1   0   0   0   1   0   0   0]
 [  0  87   0   0   0   0   0  28   0   0  15   0   0   0   0   0]
 [  0   8  17   0   0   0   0  55   1   0   0  12   1   0   0   0]
 [  0  76   4   0   0   0   0  68  53   0   1   0  35   0   0   0]
 [  0  46   0   0   0   0   0  28   6   0   0   0   1   2   0   0]
 [  0  33   0   0  

In [26]:
print(classification_report(y_test,y_pred_original,target_names=le.classes_))

                    precision    recall  f1-score   support

    cabernet franc       1.00      0.19      0.32        69
cabernet sauvignon       0.48      0.87      0.62       720
        chardonnay       0.58      0.58      0.58       298
          grenache       1.00      0.12      0.21        50
            malbec       1.00      0.07      0.13        59
            merlot       0.96      0.16      0.27       174
          nebbiolo       1.00      0.16      0.27       101
             other       0.36      0.58      0.45       562
        pinot noir       0.46      0.53      0.49       224
          riesling       0.00      0.00      0.00        44
        sangiovese       0.83      0.12      0.20       130
   sauvignon blanc       1.00      0.13      0.23        94
      shiraz syrah       0.47      0.15      0.23       237
       tempranillo       1.00      0.02      0.05        83
  touriga nacional       0.93      0.33      0.49        76
         zinfandel       0.00      0.00

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
print("Test accuracy: %.3f" % classifier_scv_original.score(X_test, y_test))

Test accuracy: 0.473


**2. SVC con escalado y PCA**

In [80]:
#normalizamos los datos
scaler=StandardScaler()

scaler.fit(X) # calculo la media para poder hacer la transformacion
X_scaled=scaler.transform(X)# Ahora si, escalo los datos y los normalizo

In [81]:
pca = PCA(n_components=1000, random_state=22)
pca.fit(X_scaled)
X = pca.transform(X_scaled)


In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [83]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6914, 1000), (2964, 1000), (6914,), (2964,))

In [44]:
param_range=[0.001,0.01,0.1,1,10]
param_grid =[
              {'kernel':['rbf'],
               'C':param_range,
               'gamma':param_range
              }
             ]

In [49]:
gs_svc = GridSearchCV(SVC(),
                      param_grid=param_grid,
                      scoring='accuracy',
                      cv=3,
                      verbose=2,
                      n_jobs=-1)

In [50]:
model_svc = gs_svc.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


In [51]:
model_svc.best_estimator_

SVC(C=10, gamma=0.01)

In [52]:
model_svc.best_score_

0.43737396561420416

In [84]:
classifier_svc_pca = SVC(C=10,gamma=0.01,kernel='rbf')

In [85]:
classifier_svc_pca.fit(X_train, y_train)

SVC(C=10, gamma=0.01)

In [86]:
y_pred_pca = classifier_svc_pca.predict(X_test)

In [87]:
result_pca = confusion_matrix(y_test, y_pred_pca)

print('Confusion Matrix - Clasificador SVC Escalado y PCA optimizado')

print(result_pca)
print('Accuracy:', accuracy_score(y_test, y_pred_pca))

Confusion Matrix - Clasificador SVC Escaladao y PCA optimizado
[[ 38   0   0   0   0   0   0  31   0   0   0   0   0   0   0   0]
 [  0 368   0   0   0   0   0 352   0   0   0   0   0   0   0   0]
 [  0   0  30   0   0   0   0 268   0   0   0   0   0   0   0   0]
 [  0   0   0  17   0   0   0  33   0   0   0   0   0   0   0   0]
 [  0   0   0   0   8   0   0  51   0   0   0   0   0   0   0   0]
 [  0   1   0   0   0  71   0 102   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  50  51   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0 562   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0 187  37   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  37   0   7   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  91   0   0  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  77   0   0   0  17   0   0   0   0]
 [  0   0   0   0   0   0   0 162   0   0   0   0  75   0   0   0]
 [  0   0   0   0   0   0   0  59   0   0   0   0   0  24   0   0]

In [88]:
print(classification_report(y_test,y_pred_pca,target_names=le.classes_))

                    precision    recall  f1-score   support

    cabernet franc       1.00      0.55      0.71        69
cabernet sauvignon       1.00      0.51      0.68       720
        chardonnay       1.00      0.10      0.18       298
          grenache       1.00      0.34      0.51        50
            malbec       1.00      0.14      0.24        59
            merlot       1.00      0.41      0.58       174
          nebbiolo       1.00      0.50      0.66       101
             other       0.26      1.00      0.41       562
        pinot noir       1.00      0.17      0.28       224
          riesling       1.00      0.16      0.27        44
        sangiovese       1.00      0.30      0.46       130
   sauvignon blanc       1.00      0.18      0.31        94
      shiraz syrah       1.00      0.32      0.48       237
       tempranillo       1.00      0.29      0.45        83
  touriga nacional       1.00      0.41      0.58        76
         zinfandel       1.00      0.05

In [89]:
print("Test accuracy: %.3f" % classifier_svc_pca.score(X_test, y_test))

Test accuracy: 0.464


In [90]:
# Validacion anidada
scores = cross_val_score(classifier_svc_pca,X_train,y_train, scoring='accuracy',cv=5)

In [91]:
print('CV accuracy: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))

CV accuracy: 0.456 +/- 0.009


**Balanceando las Clases**

In [60]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(sampling_strategy="not majority")
X_res, y_res = ros.fit_resample(X,y)

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size = 0.3, random_state=42)

In [61]:
classifier_svc_balanced = SVC(C=10,gamma=0.01,kernel='rbf')

In [62]:
classifier_svc_balanced.fit(X_train, y_train)

SVC(C=10, gamma=0.01)

In [63]:
y_pred_balanced= classifier_svc_balanced.predict(X_test)

In [64]:
result = confusion_matrix(y_test, y_pred_balanced)

print('Confusion Matrix - SVC Balanceado')

print(result)


print('Accuracy:', accuracy_score(y_test, y_pred_balanced))

Confusion Matrix - SVC Balanceado
[[628   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 314   0   0   1   0   0 337   0   0   0   0   0   0   0   0]
 [  0   0 499   0   0   0   0 170   0   0   0   0   0   0   0   0]
 [  0   0   0 685   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0 663   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0 634   0  20   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0 673   0   0   0   0   0   0   0   0   0]
 [  0   1   0   0   0   0   0 697   0   0   0   0   1   0   0   0]
 [  0   0   0   0   0   0   0  73 606   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0 637   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   9   0   0 608   0   0   0   0   0]
 [  0   0   0   0   0   0   0   5   0   0   0 675   0   0   0   0]
 [  0   0   0   0   0   0   0  64   0   0   0   0 600   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0 676   0   0]
 [  0   0   0   0   0   0   

In [65]:
print(classification_report(y_test,y_pred_balanced,target_names=le.classes_))

                    precision    recall  f1-score   support

    cabernet franc       1.00      1.00      1.00       628
cabernet sauvignon       1.00      0.48      0.65       652
        chardonnay       1.00      0.75      0.85       669
          grenache       1.00      1.00      1.00       685
            malbec       1.00      1.00      1.00       663
            merlot       1.00      0.97      0.98       654
          nebbiolo       1.00      1.00      1.00       673
             other       0.51      1.00      0.67       699
        pinot noir       1.00      0.89      0.94       679
          riesling       1.00      1.00      1.00       637
        sangiovese       1.00      0.99      0.99       617
   sauvignon blanc       1.00      0.99      1.00       680
      shiraz syrah       1.00      0.90      0.95       664
       tempranillo       1.00      1.00      1.00       676
  touriga nacional       1.00      1.00      1.00       694
         zinfandel       1.00      1.00

In [67]:
print("Test accuracy: %.3f" % classifier_svc_balanced.score(X_test, y_test))

Test accuracy: 0.936


In [ ]:
# Validacion anidada
scores = cross_val_score(classifier_svc_balanced,X_train,y_train, scoring='accuracy',cv=5)


In [69]:
print('CV accuracy: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))

CV accuracy: 0.919 +/- 0.004


In [93]:
# usamos el modulo joblib para guardar y cargar
import joblib
joblib.dump(classifier_svc_balanced, 'classifier_svc_balanced.joblib') 
#eje = joblib.load('filename.joblib')
#b=eje.predict(X[0:5])
#print(b)

['classifier_svc_balanced.joblib']